In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import sqlalchemy as sal
from sqlalchemy.engine import URL
import plotly.express as px

In [2]:
DRIVER_NAME = 'ODBC Driver 17 for SQL Server'
SERVER_NAME = 'SRVSQLC03'
DATABASE_NAME = 'UPME_STA'
USER = 'USER_ETL_DWH_STA'
PASSWORD = 'D42C06A38AAD8067BE1C9640A5257666'

connection_url = URL.create(
    "mssql+pyodbc",
    username=USER,
    password=PASSWORD,
    host=SERVER_NAME,
    port=1433,
    database=DATABASE_NAME,
    query={
        "driver": DRIVER_NAME
    },
)

try:
    engine = sal.create_engine(connection_url)
    conn = engine.connect()
    print("Conexion exitosa")
except Exception as ex:
    print("No se pudo conectar a la base de datos")
    print(ex)

Conexion exitosa


In [4]:
df_enficc = pd.read_sql("""SELECT date, value FROM dbo.daily_entity t1
WHERE id = 118
AND date >= '2010-01-01';""", conn)
# order by date
df_enficc = df_enficc.sort_values(by='date').reset_index(drop=True)
# rename value column
df_enficc.rename(columns={'value': 'enficc'}, inplace=True)
df_enficc.head()

,date,enficc
0,2010-01-01,178824048.0
1,2010-01-02,178824048.0
2,2010-01-03,178824048.0
3,2010-01-04,178824048.0
4,2010-01-05,178824048.0


In [5]:
df_demanda = pd.read_sql("""SELECT * FROM dbo.hourly_entity t1
WHERE id = 4
AND date >= '2010-01-01';""", conn)
# order by date
df_demanda = df_demanda.sort_values(by='date')
# set date index
df_demanda = df_demanda.set_index('date')
# drop id column
df_demanda = df_demanda.drop('id', axis=1)
df_demanda["demanda_comercial"] = df_demanda.sum(
    axis=1, numeric_only=True
)
df_demanda = df_demanda[["demanda_comercial"]].reset_index()
df_demanda.head()

,date,demanda_comercial
0,2010-01-01,115440933.5
1,2010-01-02,127698274.5
2,2010-01-03,123654810.5
3,2010-01-04,145964633.5
4,2010-01-05,152148028.5


In [6]:
df_oef = pd.read_sql("""SELECT date, SUM(value) as oef FROM dbo.daily_entity t1
WHERE id = 132
AND date >= '2010-01-01'
GROUP BY date;""", conn)
# order by date
df_oef = df_oef.sort_values(by='date').reset_index(drop=True)
df_oef.head()

,date,oef
0,2010-01-01,1.197586e+08
1,2010-01-02,1.315751e+08
2,2010-01-03,1.272778e+08
3,2010-01-04,1.476147e+08
4,2010-01-05,1.521460e+08


In [7]:
df = pd.merge(df_enficc, df_oef, on="date")
df = pd.merge(df, df_demanda, on="date")
df['demanda_comercial'] = df['demanda_comercial'].fillna(0)
df['enficc'] = df['enficc'].fillna(0)
df['oef'] = df['oef'].fillna(0)
df['demanda_comercial'] = df['demanda_comercial'] / 1000000
df['enficc'] = df['enficc'] / 1000000
df['oef'] = df['oef'] / 1000000

df.head()


,date,enficc,oef,demanda_comercial
0,2010-01-01,178.824048,119.758573,115.440933
1,2010-01-02,178.824048,131.575135,127.698274
2,2010-01-03,178.824048,127.277800,123.654810
3,2010-01-04,178.824048,147.614725,145.964633
4,2010-01-05,178.824048,152.146045,152.148029


In [8]:
# resample df by month
import pandas as pd
df = df.set_index('date')
df.index = pd.to_datetime(df.index)
df = df.resample('M').mean()
df.head()

,enficc,oef,demanda_comercial
date,,,
2010-01-31,178.824048,150.824378,148.960108
2010-02-28,178.824048,159.207604,157.560910
2010-03-31,178.824048,158.029521,157.700503
2010-04-30,178.824048,159.389644,153.593789
2010-05-31,178.824048,161.090067,154.350038


In [9]:
df['relacion_demnda_oef'] = df['demanda_comercial']/df['oef']
df['relacion_demnda_enficc'] = df['demanda_comercial']/df['enficc']
df.head()

,enficc,oef,demanda_comercial,relacion_demnda_oef,relacion_demnda_enficc
date,,,,,
2010-01-31,178.824048,150.824378,148.960108,0.987639,0.832998
2010-02-28,178.824048,159.207604,157.560910,0.989657,0.881095
2010-03-31,178.824048,158.029521,157.700503,0.997918,0.881875
2010-04-30,178.824048,159.389644,153.593789,0.963637,0.858910
2010-05-31,178.824048,161.090067,154.350038,0.958160,0.863139


In [11]:
df['relacion_demnda_oef'].to_list()

[0.9876394629668174,
 0.9896569428298689,
 0.9979179957567316,
 0.9636371945498801,
 0.9581598726082712,
 0.9545584983347376,
 0.944022802358903,
 0.9578234032878413,
 0.9498331381164506,
 0.9600786456632823,
 0.9574128212519336,
 0.8692081375551012,
 0.8657508985802489,
 0.8576143147786207,
 0.8759567330832827,
 0.8349971747799948,
 0.8462291332080709,
 0.858636307065182,
 0.8419672353903989,
 0.8661146493848296,
 0.8707693681757903,
 nan,
 nan,
 nan,
 0.8290828554363088,
 0.8240832200594806,
 0.8451220953757018,
 0.8132046689740055,
 0.8336880483159598,
 0.8522517658574151,
 0.8524397382129398,
 0.8435602416113316,
 0.8651297118293511,
 0.8368512959340225,
 0.8559662942517485,
 0.8350652714289616,
 0.8589130939426101,
 0.8362418344716682,
 0.8346432867380499,
 0.8729799796767375,
 0.8562991333483279,
 0.8394186424185937,
 0.8531414708252187,
 0.8317370340420775,
 0.8303813571585253,
 0.8328686414830394,
 0.8285701842073159,
 0.9638549892253379,
 0.9778287513546388,
 0.961957159516158